# Libraries

In [1]:
import pandas as pd
import numpy  as np
import string 
import re
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from collections import Counter
from wordcloud import WordCloud
from nltk.corpus import stopwords
# import nltk
# from nltk.corpus import wordnet
# from nltk.stem import WordNetLemmatizer
# #from nltk.tokenize.treebank import TreebankWordDetokenizer
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
#from nltk.corpus import stopwords

from gensim.utils import simple_preprocess
from gensim.models import Word2Vec
import gensim.downloader as api

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import spacy
import pickle
import warnings
warnings.filterwarnings('ignore')


from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping

print('Done')


2022-01-08 20:24:26.395505: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-08 20:24:26.395524: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Done


# Data

## Load data

In [2]:
pd.set_option('max_colwidth', None) 
df = pd.read_csv("~/Downloads/banData/MeTooHate.csv")[["text", "category"]]
df.dropna(inplace = True)

In [3]:
df.reset_index(inplace = True)

In [4]:
df["category"].value_counts()

0    708536
1     95102
Name: category, dtype: int64

## Help functions for initial data prep

In [5]:
def sub_sample_split_X_y( df = df, N = 100, subsample = True, undersample = True, prop = 0.5):
    # undersample
    if undersample == True:
        df1 = df[df["category"]== 1]
        df1 = df[df["category"]== 1].dropna()
        df0 = df[df["category"]== 0].dropna()
        if prop ==  0.5:
            df = pd.concat([df0.sample(df1.shape[0]), df1], axis = 0)
        else:
            df0_n = round((df1.shape[0]*prop)/(1-prop))
            df = pd.concat([df0.sample(df0_n), df1], axis = 0)
    #shuffle rows
    df = df.sample(frac=1)
    df.shape
    df.reset_index(inplace = True)
    
    if subsample == True:
        df = df.head(int(f"{N}"))
   # print(f"Number of obs in categories {df["category"].value_counts()}")

    y = df["category"]
    X = df.drop(columns = ["category"]) 
    return X,y

In [6]:
# X,y = sub_sample_split_X_y( df = df, N = 100, subsample = True, undersample = True, prop = 0.6)
# pd.DataFrame(y).value_counts()

In [7]:
def clean_data(data):
    
    #Removing URLs with a regular expression
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    data = url_pattern.sub(r'', data)

    # Remove Emails
    data = re.sub('\S*@\S*\s?', '', data)
   
    # tokenize + remove scpecial characters + set to lower case
    data = text_to_word_sequence(data) 
    
    # Remove stopwords
    stop_words = set(stopwords.words('english')) 
    data = [w for w in data if not w in stop_words]         
    
    # Remove digits
    data = ' '.join(word for word in data if not word.isdigit())
    
    
    return text_to_word_sequence(data)

def apply_data_cleaning(X, text, drop_text = False):
    ln = X.shape[0]
    sentences = []
    for i in range(ln):
        tmp = X.iloc[i][f'{text}']
        tmp_clean = clean_data(tmp)
        sentences.append(tmp_clean)
    X["sentences"] = sentences
    if drop_text == True:
        X.drop(columns = f'{text}', inplace = True)
    return list(X.sentences)

## Clean and train_test_split data

In [8]:
# X, y = sub_sample_split_X_y(df=df, N = 100, subsample = True)
# X = apply_data_cleaning(X = X, text = "text", drop_text = True)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=42)

# Embedding help functions

In [9]:
def embed_sentence(word2vec, sentence):
    """
    Function to convert a sentence (list of words) into a matrix representing 
    the words in the embedding space, training w2v on X_train
    """
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)
   

def embed_sentence_with_TF(word2vec, sentence):
    """
    Function to convert a sentence (list of words) into a matrix representing 
    the words in the embedding space, using transfer learning
    """
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
        
    return np.array(embedded_sentence)



def embedding(word2vec, sentences, TF_learning = True):
    embed = []
    """
    Function that converts a list of sentences into a list of matrices
    """
    if TF_learning == True:
        
        for sentence in sentences:
            embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
            embed.append(embedded_sentence)
    else:
        for sentence in sentences:
            embedded_sentence = embed_sentence(word2vec, sentence)
            embed.append(embedded_sentence)  

    return embed

def select_TF_model(index_list = 4):
    """ 
    Returns word2vec for a selected w2v model
    index_list: Index of the list of existing pretrained models in "api.info()['models'].keys()"
    default "glove-wiki-gigaword-50"
    """
    _list = list(api.info()['models'].keys())
    _model = _list[int(f"{index_list}")]  
    _model
    word2vec_transfer = api.load(f"{_model}")
    print(f"TF model selected: {_model}")
    return word2vec_transfer

# Embedding- selection and tuning

In [10]:
# # vary hyerparameters and check output 5.2.1
# word2vec = Word2Vec(sentences = X_train, vector_size=50, min_count = 5, window = 5)
# embedding_dim = word2vec.wv.vector_size

In [11]:
# # choose choose pretrained embedding
# word2vec = select_TF_model(index_list = 4) # default glove-wiki-gigaword-50
# # list(api.info()['models'].keys()) to see alla available ones
# embedding_dim = len(word2vec_transfer['dog'])

In [12]:
#embedding_dim

### Final data prep- embedding and padding

In [13]:
# # Embed the training and test sentences
# X_train_embed = embedding(word2vec, X_train)
# X_test_embed = embedding(word2vec, X_test)


# # Pad the training and test embedded sentences
# X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=200)
# X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=200)


# for X in [X_train_pad, X_test_pad]:
#     assert type(X) == np.ndarray
#     assert X.shape[-1] ==  embedding_dim 


# assert X_train_pad.shape[0] == len(X_train)
# assert X_test_pad.shape[0] == len(X_test)

# Run Model

## Fetch data

In [14]:
## select N
#def sub_sample_split_X_y(df=df, N = 100, subsample = True, undersample = True)
X, y = sub_sample_split_X_y(df=df, N = 100000, subsample = True, undersample = True, prop = 0.7)
X = apply_data_cleaning(X = X, text = "text", drop_text = True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=42)

In [15]:
print(pd.DataFrame(y_train).value_counts())
print(f"Proportion of 0: {pd.DataFrame(y_train).value_counts()[0]/len(y_train)}")
print(f"Proportion of 1: {pd.DataFrame(y_train).value_counts()[1]/len(y_train)}")

category
0           55914
1           24086
dtype: int64
Proportion of 0: 0.698925
Proportion of 1: 0.301075


## Embedding

### If w2v trained on X_train

In [16]:
# # # vary hyerparameters and check output 5.2.1
# word2vec = Word2Vec(sentences = X_train, vector_size=50, min_count = 5, window = 5)
# embedding_dim = word2vec.wv.vector_size

# # Embed the training and test sentences
# X_train_embed = embedding(word2vec.wv, X_train)
# X_test_embed = embedding(word2vec.wv, X_test)

In [17]:
# # # choose choose pretrained embedding

# word2vec_transfer = select_TF_model(index_list = 4) # default glove-wiki-gigaword-50 # list(api.info()['models'].keys()) to see alla available ones
# embedding_dim = len(word2vec_transfer['dog'])

# # Embed the training and test sentences
# X_train_embed = embedding(word2vec_transfer, X_train)
# X_test_embed = embedding(word2vec_transfer, X_test)


In [18]:
# print(f"'glove-wiki-gigaword-50': vocab_size {len(select_TF_model(index_list = 4))}")

In [19]:
# print(f"'glove-twitter-50': vocab_size {len(select_TF_model(index_list = 9))}")

In [20]:
# w2v_type = input(f"Choose embedding type. If transfer learning, enter yes else no   ")
# TF = input(f"If transfer learning, select model from this list {list(api.info()['models'].keys())}")               

### Pad sequence

In [21]:
# # Pad the training and test embedded sentences
# X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=200)
# X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=200)


# for X in [X_train_pad, X_test_pad]:
#     assert type(X) == np.ndarray
#     assert X.shape[-1] ==  embedding_dim 


# assert X_train_pad.shape[0] == len(X_train)
# assert X_test_pad.shape[0] == len(X_test)

## Train model

### Select embedding type

In [22]:
list_TF = list(api.info()['models'].keys())
# w2v_type = input(f"Choose embedding type. If transfer learning, enter yes else no   ")
# TF = input(f"If transfer learning, select model from this list {list_TF}")  


In [23]:
# #def select_TF_embedding(X_train, X_test):
# index_list = list_TF.index('glove-twitter-50')
# index_list

9

In [25]:
# word2vec_transfer = select_TF_model(index_list = index_list)

In [ ]:
print(f"Vocab_size: {len(word2vec_transfer.key_to_index)}")

### Embed and pad

In [1]:
# # choose choose pretrained embedding
word2vec_transfer = select_TF_model(index_list = 4)index_list) # 4 default glove-wiki-gigaword-50 # list(api.info()['models'].keys()) to see alla available ones
embedding_dim = len(word2vec_transfer['dog'])

# Embed the training and test sentences
X_train_embed = embedding(word2vec_transfer, X_train)
X_test_embed = embedding(word2vec_transfer, X_test)

# Pad the training and test embedded sentences
X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=200)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=200)

# Test dim of tensor ok w.r.t to input X
for X in [X_train_pad, X_test_pad]:
    assert type(X) == np.ndarray
    assert X.shape[-1] ==  embedding_dim 


assert X_train_pad.shape[0] == len(X_train)
assert X_test_pad.shape[0] == len(X_test)

In [ ]:
def init_model():
    model = Sequential()
    model.add(layers.Masking())
    model.add(layers.LSTM(20, activation='tanh'))
    model.add(layers.Dense(15, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=["Recall"]) #['recall']) # tp/(tp+fn) metrics=[tf.keras.metrics.Recall()]
    
    return model

model = init_model()


In [ ]:
# model.summary()

**!!! remember to initialize before every new run**

In [ ]:
es = EarlyStopping(patience=10, restore_best_weights=True)
%%time
history = model.fit(X_train_pad, y_train, 
          batch_size = 16,  # low --> optimizes on smaller ss --> faster --> but to generalize 
          epochs=100,
          validation_split=0.3,
          callbacks=[es]
         )


In [ ]:
res = model.evaluate(X_test_pad, y_test, verbose=1)
print(f'The Recall evaluated on the test set is of {res[1]*100:.3f}%')

In [ ]:
#history.history

In [ ]:
def plot_loss_recall(history, title=None):
    fig, ax = plt.subplots(1,2, figsize=(20,7))
    
    # --- LOSS --- 
    
    ax[0].plot(history.history['loss'])
    ax[0].plot(history.history['val_loss'])
    ax[0].set_title('Model loss')
    ax[0].set_ylabel('Loss')
    ax[0].set_xlabel('Epoch')
    ax[0].set_ylim((0,3))
    ax[0].legend(['Train', 'Test'], loc='best')
    ax[0].grid(axis="x",linewidth=0.5)
    ax[0].grid(axis="y",linewidth=0.5)
    
    # --- ACCURACY
    
    ax[1].plot(history.history['recall'])
    ax[1].plot(history.history['val_recall'])
    ax[1].set_title('Model Recall')
    ax[1].set_ylabel('recall')
    ax[1].set_xlabel('Epoch')
    ax[1].legend(['Train', 'Test'], loc='best')
    ax[1].set_ylim((0,1))
    ax[1].grid(axis="x",linewidth=0.5)
    ax[1].grid(axis="y",linewidth=0.5)
    
    if title:
        fig.suptitle(title)

In [ ]:
plot_loss_recall(history, title=None)

In [ ]:
print(f"sample size: {len(y)}")
print(f"TF used: {list_TF[index_list]}")
print(f"Proportion of 0: {pd.DataFrame(y_train).value_counts()[0]/len(y_train)}")
print(f"Proportion of 1: {pd.DataFrame(y_train).value_counts()[1]/len(y_train)}")

## Extras 

## Checking embedding

### Vocabulary size, w2v, similar words

In [ ]:
word2vec = Word2Vec(sentences = X_train, vector_size=50, min_count = 5, window = 5)
vocab = word2vec.wv.key_to_index 
print(len(vocab))
#print(vocab)
print(word2vec.wv["women"])
word2vec.wv.most_similar('women')

### mini-batch

The smaller the batch, the more stochastic the process is, the faster it may converge
The larger the batch, the better it generalizes, but the more computation-intensive it becomes